In [1]:
import torch
print(torch.__version__)

2.0.1+cpu


In [9]:
#pytorch = 1.9 torchtext = 0.10

In [9]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, BatchSampler

In [10]:
from PIL import Image
import torch
import torchtext
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np


OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다

In [7]:
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow

import nltk, re, string, collections
from nltk.util import ngrams
import collections
%matplotlib inline
BATCH_SIZE = 64
EPOCH = 2

In [5]:
import pickle #모델 pickle형식으로 저장하기
my_model = net.load_state_dict(torch.load(r"C:\Users\Park se woong\Desktop\skku\best_model_33.pt", map_location=torch.device('cpu')))

pickle.dump(my_model, open('model.pkl','wb'))

NameError: name 'net' is not defined

In [4]:
#image file is truncated -> 오류해결코드
#image 파일이 잘려있으면 나오는 오류
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
import tensorflow as tf

# 모델 로드
model = tf.keras.models.load_model(r"C:\Users\Park se woong\Desktop\skku\last_model.h5")

# 모델 요약 정보 출력
model.summary()


OSError: No file or directory found at C:\Users\Park se woong\Desktop\skku\last_model.h5

# 준비

## Load the data

In [5]:
from collections import OrderedDict

In [6]:
# IMG_ROOT = "data/j_images"
JSON_ROOT = "../data/shoe_json"
#img_paths = glob.glob(os.path.join("../data/j_images", "*/*.png"))

# 학습용 
# img_paths = glob.glob(os.path.join("../data/j_images", "*/posts/*.jpg"))
#img_paths = glob.glob(os.path.join("../data/imgs", "*/*")) #모든 이미지
img_paths2 = glob.glob(os.path.join("../data/shoe_imgs_final", "*/*")) #크림 제품 이미지

NameError: name 'glob' is not defined

In [ ]:
#img_paths += glob.glob(os.path.join("../data/j_images", "*/posts/*.jpeg"))

In [8]:
# file_paths = glob.glob(os.path.join("../data/imgs", "*/*.png"))
# len(file_paths)

2548

In [10]:
len(img_paths)

NameError: name 'img_paths' is not defined

In [9]:
len(img_paths2)

44400

In [8]:
# cropped_img_list = []
# for crop in img_paths:
#   if 'cropped' in crop:
#     cropped_img_list.append(crop)

In [1]:
# print(img_paths)
# d = {}
caption_data = {} # {'product_name': [img_paths...]}

# for i, img_path in tqdm(enumerate(img_paths)):
#     name = img_path.split("/")[3]
#     try:
#         with open(os.path.join(JSON_ROOT, name + ".json"), "r") as f:
#             captions = json.load(f)
#             caption = captions[0]
#             if not caption_data.get(caption):
#                 caption_data[caption] = []
#             caption_data[caption].append(img_path)
#     except:
#         continue

##caption_data = {제품명 : [이미지 경로들]}
for i, img_path2 in tqdm(enumerate(img_paths2)):
    name = img_path2.split("/")[3]
    try:
        with open(os.path.join(JSON_ROOT, name + ".json"), "r") as f:
            captions = json.load(f)
            caption = captions[0]
            if not caption_data.get(caption):
                caption_data[caption] = []
            caption_data[caption].append(img_path2)
    except:
        continue

len(caption_data)

NameError: name 'tqdm' is not defined

In [ ]:
caption_data

In [17]:
caption_data = OrderedDict(caption_data)

In [18]:
len(caption_data)

34721

## Create torch dataset 

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)


class JordanDataset(Dataset):
    """
    create processed data
    
    self.processed_cache: {<[img_path]>: <processed image binary>}
    self.img_paths_set: img_path의 리스트
    self.path2label: {img_path: label(index)}
    
    self.labels: [catpion에 해당하는 index리스트]
    self.captions: [catpion 내용, ...]
    """
    def __init__(self, data, preprocess, size_per_caption=0, on_memory_image=False):
        """
        :param data: dict {<product Name>: [img_paths...]}   
        :param: preprocess: func clip[Vit-B/32].preprocess # model, preprocess = clip.load("ViT-B/32", device=device, jit=False) 
        :param: size_per_caption: 상품당 스타일 이미지의 개수
        :param: on_memory_image: 이미지를 date 준비과정에 메모리에 올려놓을지 여부
        
        """
        self.on_memory_image = on_memory_image

        self.preprocess = preprocess
        self.img_paths = []
        self._captions = []
        
        self.path2label = {}
        self.label_list = []
        
        self.processed_cache = {}
        
        if size_per_caption < 0:
            raise ValueError('size_per_caption must be positive integer.')
        
        for idx, (caption, img_paths) in tqdm(enumerate(data.items())):
            max_size = len(img_paths)
#             num = img_paths.split('/')[-2].split('.')[0].split('_')[0]
            
            if size_per_caption > 0:
                max_size = min(size_per_caption, max_size) 
            
            for img_path in img_paths:
                num = img_path.split('/')[-2]
                self.img_paths.append(img_path)
                self._captions.append(caption)
                self.path2label[img_path] = int(num)
                
                if on_memory_image:
                    self.processed_cache[img_path] = self.preprocess(Image.open(img_path))
            
            self.label_list.append(caption)
        # for img_path in data:
        #     self.processed_cache[img_path] = self.preprocess(Image.open(img_path))
        
    def __len__(self):
        return len(self._captions)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]

        # image = self.processed_cache[img_path]
        if self.on_memory_image:
            image = self.processed_cache[img_path]
        else:
            image = self.preprocess(Image.open(img_path))
        
        caption = self._captions[idx]
        label = self.path2label[img_path]
        return image, caption, label
    
    @property
    def captions(self):
        return self._captions
    
    @property
    def labels(self):
        return [self.path2label[img_path] for img_path in self.img_paths]
























































































































































































































































































































































































































































































































































































































































































































































100%|███████████████████████████████████████| 338M/338M [01:51<00:00, 3.17MiB/s]


In [14]:
import math

In [23]:
def train_test_split(data, train_size=0.9):
    d_train = OrderedDict()   
    d_test = OrderedDict()
    
    for img_path, prod_name in data.items():
        train_end_idx = math.ceil(len(prod_name) * train_size)
        d_train[img_path] = prod_name[:train_end_idx]
        d_test[img_path] = prod_name[train_end_idx:]
    
    return d_train, d_test
    

In [16]:
# train_size = 0.1로 할시 test_data부족으로인해 오류남
d_train, d_test = train_test_split(caption_data, train_size=0.8)

In [22]:
train_dataset = JordanDataset(d_train, preprocess)
test_dataset = JordanDataset(d_test, preprocess)
# len(train_dataset), len(test_dataset), train_dataset[0]

Exception ignored in: <function tqdm.__del__ at 0x00000217D308A170>
Traceback (most recent call last):
  File "c:\Users\Park se woong\.conda\envs\gpu_env2\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\Park se woong\.conda\envs\gpu_env2\lib\site-packages\tqdm\notebook.py", line 282, in close
    self.disp(bar_style='success', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [19]:
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, labels, n_classes, n_samples): #초기화
        self.labels = labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.n_dataset = len(self.labels)
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while (self.count + self.batch_size) < self.n_dataset:
            classes = np.random.choice(self.labels_set, self.n_classes, replace=True)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                                self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                        class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices #여러개 나눠서 출력
            self.count += self.batch_size
            # self.count += self.n_classes * self.n_samples

    def __len__(self):
        return self.n_dataset // self.batch_size
    


In [20]:
train_labels = torch.tensor(train_dataset.labels) # label list
train_sampler = BalancedBatchSampler(train_labels, BATCH_SIZE, 1) # label_list -> batch label
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler) # ready for dataloader

test_labels = torch.tensor(test_dataset.labels)
test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)

NameError: name 'train_dataset' is not defined

In [21]:
len(train_dataset.labels)

NameError: name 'train_dataset' is not defined

In [8]:
# parameter를 float32로 변환
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

NameError: name 'device' is not defined

In [32]:
from ipywidgets import FloatProgress

In [38]:
EPOCH = 2
#image rotate or noise

In [39]:
best_te_loss = 1e5
best_ep = -1
for epoch in range(EPOCH):
    print(f"running epoch {epoch+33}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_dataloader, leave=False)
    for batch in pbar:
        step += 1
        optimizer.zero_grad()

        images, texts, _ = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
#         print(images.shape, texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step
    
    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, _ = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(BATCH_SIZE).to(device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step
        
    if te_loss < best_te_loss:
        best_te_loss = te_loss
        best_ep = epoch
        torch.save(model.state_dict(), f"best_model_{epoch+33}.pt")
    print(f"epoch {epoch+33}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "last_model.pt")

running epoch 33, best test loss 100000.0 after epoch -1


  0%|          | 0/684 [00:00<?, ?it/s]

/home/psw1022s/my_env/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 33, tr_loss 0.034192509818495365, te_loss 0.77294921875
running epoch 34, best test loss 0.77294921875 after epoch 0


  0%|          | 0/684 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

epoch 34, tr_loss 0.020328805286284776, te_loss 0.70556640625


In [4]:
model.load_state_dict(torch.load("best_model_33.pt"))
NUM_NEG = 127
NUM_TEST = 1000

NameError: name 'model' is not defined

In [29]:
len(test_dataloader)

2

In [26]:
NUM_TEST = min(NUM_TEST, len(test_dataloader))

In [31]:
iter(test_dataloader)

In [ ]:
test_dataset[8][0]

In [27]:
test_dataloader = DataLoader(test_dataset, shuffle=True)

test_dataset

In [ ]:
n_correct = 0
step = 0
test_loader = tqdm(test_dataloader)

for data in test_loader:
    step += 1
    images, texts, _ = data
    images = images.to(device)
    texts = clip.tokenize(texts).to(device)
#         print(images.shape, texts.shape)
    logits_per_image, logits_per_text = model(images, texts)
    ground_truth = torch.arange(BATCH_SIZE).to(device)
    n_correct +=1
    
    total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
    total_loss.backward()
    tr_loss += total_loss.item()
    if device == "cpu":
        optimizer.step()
        scheduler.step()
    else:
        convert_models_to_fp32(model)
        optimizer.step()
        scheduler.step()
        clip.model.convert_weights(model)
    pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
tr_loss /= step

step = 0
te_loss = 0
with torch.no_grad():
    model.eval()
    test_pbar = tqdm(test_dataloader, leave=False)
    for batch in test_pbar:
        step += 1
        images, texts, _ = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        te_loss += total_loss.item()
        test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
    te_loss /= step

if te_loss < best_te_loss:
    best_te_loss = te_loss
    best_ep = epoch
    torch.save(model.state_dict(), "best_model_3t")
print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "last_model_3.pt")

In [ ]:
data[0], data[1]

In [41]:
NUM_NEG

127

In [29]:
test_dataloader = DataLoader(test_dataset, shuffle=True)

In [ ]:
test_loader = iter(test_dataloader)
c=  0
for data in test_loader:
    c+= len(data[1])
    print(len(data[1]), data[1])

In [155]:
corpus = []
for txtlist in d_train.keys():
    corpus.append(txtlist)

In [152]:
for txtlist in d_train.keys():
    print(txtlist)
    break

Adidas Gazelle 85 Blue Bird


In [184]:
corpus[:10]

['Adidas Gazelle 85 Blue Bird',
 'Nike Air Max 95 Mowabb',
 'Salomon XT-6 ADV Black Racing Red',
 'Celine Z Trainer CT-01 High Top Calfskin Sneakers Black Optic White',
 'Nike Air VaporMax Utility Game Royal',
 'Alexander McQueen Oversized Sneakers Black Crocodile',
 'Nike Pre Montreal Racer Light Charcoal',
 'Jordan 4 x Union Retro Guava Ice',
 'Nike Zoom Mercurial Superfly 9 Academy TF White Baltic Blue',
 '(W) Burberry Check Print Slides Archive Beige']

In [185]:
with open('corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)

In [ ]:
!nvidia-smi

In [157]:
len(corpus)

34721

## find all_text_VECTOR

In [158]:
text_clip_list = []

for i in range(0, len(corpus), 100):
    texts = clip.tokenize(corpus[i:i+100]).to(device)

    with torch.no_grad():
        text_features = model.encode_text(texts)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        text_features = text_features.detach().cpu()

        text_clip_list.append(text_features)
    
print(len(text_clip_list))
    

348


In [1]:
import torch
from transformers import CLIPProcessor, CLIPModel

# CLIP 모델과 프로세서 불러오기
model_name = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

# 텍스트를 임베딩하는 예제
text_input = ["A cat sitting on a mat"]
tokenized_text = processor(text_input, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    text_features = model.encode_text(**tokenized_text)
    
print(text_features)

ModuleNotFoundError: No module named 'transformers'

In [159]:
text_vector =  torch.cat(text_clip_list, 0)

In [160]:
text_vector.shape

torch.Size([34721, 512])

In [178]:
import pickle
with open('text_vector.pkl', 'wb') as f:
    pickle.dump(text_vector, f)

## test image and calculate acc

## top_k acc 함수 설정

In [6]:
def accuracy(output, target, topk=(10,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    
    return res

# 전체 SIMILARITY 계산

In [161]:
sim1 = []

In [7]:
for i in test_dataset:
    image = i[0].to(device)
    image_features = model.encode_image(image.unsqueeze(0))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    image_features = image_features.detach().cpu()
    
    
    
    similarity = (100.0 * image_features @ text_vector.T).type(torch.float64)
    sim1.append(similarity)

    values, indices = similarity[0].topk(10)
    print(i[2], indices)
    for zz in indices:
        print(corpus[zz])

# image = test_dataset[0][0].to(device)
# image_features = model.encode_image(image.unsqueeze(0))
# image_features /= image_features.norm(dim=-1, keepdim=True)
# image_features = image_features.detach().cpu()
# image_features @ text_vector.T.softmax(dim=-1)
# similarity = (100.0 * image_features @ text_vector.T).type(torch.float32).softmax(dim=1)
# # values, indices = similarity[0].topk(5)


NameError: name 'test_dataset' is not defined

In [183]:
test1_image = '../data/shoe_imgs_final/38367/0.png'
test2_image = '../data/j_images/84/posts/703747/0.jpeg'

img = preprocess(Image.open(test1_image)).to(device)

image_features1 = model.encode_image(img.unsqueeze(0))
image_features1 /= image_features1.norm(dim=-1, keepdim=True)
image_features1 = image_features1.detach().cpu()


similarity = (100.0 * image_features1 @ text_vector.T).type(torch.float32)

values, indices = similarity[0].topk(10)
print(values, indices)
for zz in indices:
    print(corpus[zz])

tensor([39.4688, 37.3438, 37.2188, 37.0938, 36.7500, 36.6250, 36.6250, 36.1562,
        36.0312, 35.8750]) tensor([ 2384, 30616, 29006, 24488, 13101, 29668, 10970, 17046, 20790, 18330])
Jordan 1 Retro Low OG Black and Dark Powder Blue
Jordan 1 Low Black Blue White
Jordan 1 Low SE Tokyo 96
Jordan 1 x Travis Scott x Fragment Retro Low OG SP Military Blue
Nike Blazer Low '77 Vintage White Blue Black
Nike Daybreak Type SE Label Maker Summit White
Nike x Undercover Daybreak Blue Jay
Jordan 1 Low SE Laser Blue
(W) Nike Daybreak SE Worldwide Pack
Nike Daybreak Type N.354 Black


In [107]:
similarity

tensor([[21.4844, 19.8750, 19.4531,  ..., 22.2344, 21.2188, 22.0625]],
       dtype=torch.float64)

In [116]:
similarity.softmax(dim=1, dtype=torch.float64)[0]

tensor([2.2229e-05, 4.4461e-06, 2.9158e-06,  ..., 4.7059e-05, 1.7043e-05,
        3.9627e-05], dtype=torch.float64)

In [132]:
similarity[0]

tensor([21.4844, 19.8750, 19.4531,  ..., 22.2344, 21.2188, 22.0625],
       dtype=torch.float64)

In [124]:
similarity.shape

torch.Size([1, 43782])

In [111]:
similarity

tensor([[21.4844, 19.8750, 19.4531,  ..., 22.2344, 21.2188, 22.0625]],
       dtype=torch.float64)

In [88]:
values

tensor([0.0012, 0.0011, 0.0011, 0.0011, 0.0010, 0.0010, 0.0010, 0.0010, 0.0009,
        0.0009])

In [98]:
similarity.softmax(dim=1, dtype=torch.float64).shape

torch.Size([1, 43782])

In [101]:
similarity

tensor([[2.2229e-05, 4.4461e-06, 2.9158e-06,  ..., 4.7059e-05, 1.7044e-05,
         3.9627e-05]])

In [90]:
similarity

torch.Size([1, 43782])

In [87]:
values[0]

tensor(0.0012)

In [53]:
len(corpus)

43782

In [68]:
test_dataset[0][0]

tensor([[[-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
         [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
         [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
         ...,
         [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
         [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
         [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923]],

        [[-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
         [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
         [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
         ...,
         [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
         [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
         [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521]],

        [[-1.4802, -1.4802, -1.4802,  ..., -1.4802, -1.4802, -1.4802],
         [-1.4802, -1.4802, -1.4802,  ..., -1

In [54]:
corpus[34038]

'../data/shoe_imgs_final/66348/1.png'

In [56]:
corpus[1551]

'../data/shoe_imgs_final/66353/1.png'

In [49]:
image_features.shape

torch.Size([1, 512])

In [50]:
text_vector.shape

torch.Size([43782, 512])

In [42]:
similarity = (100.0 * image_features @ text_vector.T).type(torch.float32).softmax(dim=1)
values, indices = similarity[0].topk(5)

In [45]:
indices

tensor([30611, 10047, 15228, 43259, 38539])

In [49]:
image_features /= image_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).type(torch.float32).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
# print("\nTop predictions:\n")
# for value, index in zip(values, indices):
#     print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

In [64]:
for data in tqdm(test_loader):
    
    image = data[0].to(device)
    name = data[2]
    with torch.no_grad():
        image_features = model.encode_image(image)
        image_features = image_features.numpy()
        

  0%|          | 0/190 [00:00<?, ?it/s]

In [30]:
len(corpus)

43782

In [ ]:
text = clip.tokenize(['']).to(device)

In [26]:
# 제가 볼게요.

n_correct = 0

test_loader = iter(test_dataloader)

for data in tqdm(test_loader):
    
    # data = next(test_loader)
    image = data[0].to(device)
    name = data[2]

    title = data[1] #제품 이름
    texts = clip.tokenize(corpus)[::10].to(device)
    
    with torch.no_grad():
#         image_features = model.encode_image(image)
        text_features = model.encode_text(texts)

        logits_per_image, logits_per_text = model(image, texts)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    
        # print(np.argsort(probs)[0, :])

    #     print("Label probs:", probs)
    #     print(np.argmax(probs))
    #     imshow(np.asarray(Image.open(img_path)))
#     print(texts[np.argsort(-probs)[0, 0]], texts[np.argsort(-probs)[0, 1]], texts[np.argsort(-probs)[0, 2]])
#     print("Label probs:", probs)
#     print(np.argmax(probs))
    if np.argmax(probs) == 0:
        n_correct +=1
    
print(f"Test precision {n_correct/NUM_TEST}")

  0%|          | 0/2 [00:00<?, ?it/s]

Test precision 0.0


In [178]:
logits_per_image

tensor([[22.1562, 21.7344, 22.0469, 22.2812, 22.1719]], device='cuda:0',
       dtype=torch.float16)

In [177]:
n_correct / 190

0.07894736842105263

In [168]:
texts

tensor([[49406,   263,   342,   264,  5783,   343, 29692,  1437,  1099, 10311,
         17222,  1042,  1449, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], device='cuda:0')

In [167]:
logits_per_image

tensor([[30.7812]], device='cuda:0', dtype=torch.float16)

In [114]:
NUM_TEST

2

In [66]:
#내가만든쿠키
n_correct = 0

test_loader = iter(test_dataloader)

for i in tqdm(range(NUM_TEST)):
    
    data = next(test_loader)
    for i in range(len(test_dataset)):
        empty = True
        while empty:
            caps = random.choice(list(d_test.keys()))
    #         image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
            image = test_dataset[i][0].unsqueeze(0).to(device)
            name = test_dataset[i][2]
            img_path = d_test[caps]
            
            pos_txt = random.choice(caps)
            break
#             if len(caps) > 0:
#                 pos_txt = random.choice(caps)
#             #         pos_txt = ' '.join(pos_txt)
#                 empty = False
    #     print(pos_txt)
        neg_i = 0
        neg_txts = []
        
    while neg_i < NUM_NEG:
        caps = random.choice(list(d_test.keys()))
        img_path = d_test[caps]
        neg_name = img_path[0].split('/')[-2]
        if neg_name == name:
            continue
        if len(caps) == 0:
            continue
        neg_txt = random.choice(caps)
        if neg_txt in neg_txts:
            continue
        neg_txts.append(neg_txt)
        neg_i += 1
#     print(name)
#     print(f"Positive caption: {pos_txt}")
#     print(f"Negative caption: {neg_txts}")
    text = clip.tokenize([pos_txt]+neg_txts).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

#     print("Label probs:", probs)
#     print(np.argmax(probs))
    if np.argmax(probs) == 0:
        n_correct +=1
print(f"Test precision {n_correct/NUM_TEST}")

  0%|          | 0/2 [00:00<?, ?it/s]

IndexError: list index out of range

In [312]:
len(test_dataset)

4930

In [41]:
def sample1Caption(img_path, corpus, model, num_cand):
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    i = 0
    txts = []
    while i < num_cand:
        txt = random.choice(corpus)
        if txt in txts:
            continue
        if len(txt.split())<5 or len(txt)>72:
            continue
        txts.append(txt)
        i += 1
 
    text = clip.tokenize(txts).to(device)

    with torch.no_grad():
        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        print(np.argmax(probs))

        print(np.argsort(-probs)[0, :3])

    return [txts[np.argsort(-probs)[0, 0]], txts[np.argsort(-probs)[0, 1]], txts[np.argsort(-probs)[0, 2]]]


In [40]:
#model.load_state_dict(torch.load("best_model.pt"))
corpus = []
for txtlist in d_train.keys():
    corpus.append(txtlist)
len(corpus), corpus[0]

(34721, 'Adidas Gazelle 85 Blue Bird')

In [ ]:
d_train

In [ ]:
caption = sample1Caption(img_path2, corpus, model, 1000)

In [37]:
captions = {}
for img_path in tqdm(d_test.keys()):
    caption = sample1Caption(img_path, corpus, model, 1000)
    print(caption)
    captions[img_path] = caption

TypeError: 'builtin_function_or_method' object is not subscriptable

In [38]:
for get_bleu in range(1,4):
    bleu_x_lst = []
    bleu_y_lst = []
    for p, caps in d_test.items():
        if not caps:
            continue
        # print(captions[p])
        bleu_x_lst.append(captions[p].split())
        splittedcaps = [x.split() for x in caps]
        bleu_y_lst.append(splittedcaps)
    BLEU = torchtext.data.metrics.bleu_score(bleu_x_lst, bleu_y_lst, max_n=get_bleu, weights=[1/get_bleu]*get_bleu)
    print(f"{get_bleu}-gram BLEU score: {BLEU}")

KeyError: 'Adidas Gazelle 85 Blue Bird'

In [39]:
sentences = list(captions.values())
BigramCtr = collections.Counter()
UnigramCtr = collections.Counter()
for sentence in sentences:
    BigramCtr.update(nltk.ngrams(sentence, 2))
    UnigramCtr.update(nltk.ngrams(sentence, 1))
# print("Unigram count:",len(BigramCtr)/len(sentences))
# print("Bigram count:",len(UnigramCtr)/len(sentences))
print("Unigram count:",len(BigramCtr))
print("Bigram count:",len(UnigramCtr))

Unigram count: 0
Bigram count: 0


In [40]:
from matplotlib import pyplot as plt

In [ ]:
for i in d_train.keys():
    for j in range(len(d_train[i])):
#         print(len(d_train[i]))
        print(d_train[i][j])

In [ ]:
seen_path = random.choice(list(d_train.keys()))
# seen_path = '../data/j_images/89214/posts/674717/0_cropped_1.jpeg'
pred_cap_seen = sample1Caption(seen_path, corpus, model, 1000)
gt_cap_seen = d_train[seen_path]
plt.imshow(Image.open(seen_path))
print(f"Some ground truth captions for this seen image: {gt_cap_seen}")
print(f"Caption sampled by fintuned CLIP for this seen image: {pred_cap_seen}")

In [ ]:
sample1Caption(seen_path, corpus, model, 1000)

In [ ]:
unseen_path = random.choice(list(d_test.keys()))
pred_cap_unseen = sample1Caption(unseen_path, corpus, model, 1000)
imshow(Image.open(unseen_path))
gt_cap_unseen = d_test[unseen_path]
# imshow(Image.open(pred_cap_unseen))
print(f"Some ground truth captions for this unseen image: {gt_cap_unseen}")
print(f"Caption sampled by fintuned CLIP for this unseen image: {pred_cap_unseen}")

In [ ]:
d_test[unseen_path]

In [ ]:
unseen_path

In [ ]:
test_image = '../data/imgs/42950/42950_3_cropped_1.jpg'
predict_label = sample1Caption(test_image, corpus, model, 1000)
imshow(Image.open(test_image))
# dd = d_test[aa]
# # imshow(Image.open(pred_cap_unseen))
# print(f"Some ground truth captions for this unseen image: {dd}")
print(f"Caption sampled by fintuned CLIP for this unseen image: {predict_label}")

In [ ]:
test_image = '../data/imgs/25063/1.jpg'
predict_label = sample1Caption(test_image, corpus, model, 10)
imshow(Image.open(test_image))
# dd = d_test[aa]
# # imshow(Image.open(pred_cap_unseen))
# print(f"Some ground truth captions for this unseen image: {dd}")
print(f"Caption sampled by fintuned CLIP for this unseen image: {predict_label}")

In [44]:
test_image = '../data/j_images/21922/posts/260756/0_cropped_0.jpeg'
predict_label = sample1Caption(test_image, corpus, model, 1000)
imshow(Image.open(test_image))
# dd = d_test[aa]
# # imshow(Image.open(pred_cap_unseen))
# print(f"Some ground truth captions for this unseen image: {dd}")
print(f"Caption sampled by fintuned CLIP for this unseen image: {predict_label}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/j_images/21922/posts/260756/0_cropped_0.jpeg'

In [42]:
test_image = '../data/shoe_imgs_final/45487/0.png'
predict_label = sample1Caption(test_image, corpus, model, 1000)
# imshow(Image.open(test_image))
# dd = d_test[aa]
# # imshow(Image.open(pred_cap_unseen))
# print(f"Some ground truth captions for this unseen image: {dd}")
print(f"Caption sampled by fintuned CLIP for this unseen image: {predict_label}")

KeyboardInterrupt: 